In [5]:
import glob
import h5py
import numpy as np 
import os
import pickle
import torch 

from omegaconf import DictConfig, OmegaConf
# from tactile_learning.deployment.load_models import load_model
from tactile_learning.datasets.tactile import TactileImage

In [55]:
# Helper script to load models
import numpy as np
import os
import torch
import torch.utils.data as data 
import torchvision.transforms as T

from collections import OrderedDict
from tqdm import tqdm 
from torch.nn.parallel import DistributedDataParallel as DDP

from holobot.robot.allegro.allegro_kdl import AllegroKDL
from tactile_learning.models.custom import TactileJointLinear, TactileImageEncoder

def load_model(cfg, device, model_path):
    # Initialize the model
    if cfg.agent_type == 'bc':
        model = TactileJointLinear(
            input_dim=cfg.tactile_info_dim + cfg.joint_pos_dim,
            output_dim=cfg.joint_pos_dim,
            hidden_dim=cfg.hidden_dim
        )
    elif cfg.agent_type == 'byol': # load the encoder
        model = TactileImageEncoder(
            in_channels=cfg.encoder.in_channels,
            out_dim=cfg.encoder.out_dim
        )
    # print('model: {}'.format(model))
    state_dict = torch.load(model_path)
    
    # Modify the state dict accordingly - this is needed when multi GPU saving was done
    new_state_dict = modify_multi_gpu_state_dict(state_dict)
    
    if cfg.agent_type == 'byol':
        new_state_dict = modify_byol_state_dict(new_state_dict)

    # Load the new state dict to the model 
    model.load_state_dict(new_state_dict)

    # Turn it into DDP - it was saved that way 
    model = DDP(model.to(device), device_ids=[0])

    return model

def modify_multi_gpu_state_dict(state_dict):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]
        new_state_dict[name] = v 
    return new_state_dict

def modify_byol_state_dict(state_dict):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if 'encoder.net' in k:
            name = k[12:] # Everything after encoder.net
            new_state_dict[name] = v
    # print(new_state_dict['encoder.net'])
    return new_state_dict


In [12]:
import glob
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle 
import torch
import torchvision.transforms as T

from omegaconf import OmegaConf
from tqdm import tqdm 

from holobot.robot.allegro.allegro_kdl import AllegroKDL
from tactile_learning.deployment.load_models import load_model
from tactile_learning.deployment.nn_buffer import NearestNeighborBuffer
from tactile_learning.models.knn import KNearestNeighbors
from tactile_learning.utils.visualization import dump_camera_image, dump_tactile_state, dump_knn_state
from tactile_learning.utils.tactile_image import get_tactile_image

class DeployVINN:
    def __init__(
        self,
        out_dir,
        robots = ['allegro', 'kinova'],
        sensor_indices = (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14),
        allegro_finger_indices = (0,1,2,3),
        use_encoder = True,
        only_states = False,
        nn_buffer_size=100,
        nn_k=20
    ):
        os.environ["MASTER_ADDR"] = "localhost"
        os.environ["MASTER_PORT"] = "29505"

        # torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
        torch.cuda.set_device(0)

        self.sensor_indices = sensor_indices 
        self.allegro_finger_indices = [j for i in allegro_finger_indices for j in range(i*3,(i+1)*3)]
        print('self.allegro_finger_indices: {}'.format(self.allegro_finger_indices))
        self.only_states = only_states # Check to add tactile info to the representation or not
        self.use_encoder = use_encoder
        self.robots = robots # Types of robots to be used in the states
        print('self.only_states: {}, self.use_encoder: {}'.format(
            self.only_states, self.use_encoder
        ))

        self.device = torch.device('cuda:0')
        self.out_dir = out_dir
        self.cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
        self.data_path = self.cfg.data_dir
        model_path = os.path.join(out_dir, 'models/byol_encoder.pt')

        self.encoder = load_model(self.cfg, self.device, model_path)
        self.encoder.eval() 

        self.resize_transform = T.Resize((self.cfg.tactile_image_size, self.cfg.tactile_image_size))

        self._load_data()
        self._get_all_representations()
        self.state_id = 0 # Increase it with each get_action

        self.kdl_solver = AllegroKDL()
        self.last_action = None
        self.buffer = NearestNeighborBuffer(nn_buffer_size)
        self.nn_k = nn_k
        self.knn = KNearestNeighbors(
            self.all_representations, # Both the input and the output of the nearest neighbors are
            self.all_representations
        )

    def _load_data(self):
        roots = glob.glob(f'{self.data_path}/demonstration_*')
        roots = sorted(roots)

        self.tactile_indices = [] 
        self.allegro_indices = [] 
        self.kinova_indices = []
        self.allegro_action_indices = [] 
        self.allegro_actions = [] 
        self.tactile_values = [] 
        self.allegro_tip_positions = []
        self.kinova_states = []

        for root in roots:
            # Load the indices
            with open(os.path.join(root, 'tactile_indices.pkl'), 'rb') as f:
                self.tactile_indices += pickle.load(f)
            with open(os.path.join(root, 'allegro_indices.pkl'), 'rb') as f:
                self.allegro_indices += pickle.load(f)
            with open(os.path.join(root, 'allegro_action_indices.pkl'), 'rb') as f:
                self.allegro_action_indices += pickle.load(f)
            with open(os.path.join(root, 'kinova_indices.pkl'), 'rb') as f:
                self.kinova_indices += pickle.load(f)

            # Load the data
            with h5py.File(os.path.join(root, 'allegro_fingertip_states.h5'), 'r') as f:
                self.allegro_tip_positions.append(f['positions'][()][:, self.allegro_finger_indices])
            with h5py.File(os.path.join(root, 'allegro_commanded_joint_states.h5'), 'r') as f:
                self.allegro_actions.append(f['positions'][()]) # Positions are to be learned - since this is a position control
            with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
                self.tactile_values.append(f['sensor_values'][()][:,self.sensor_indices,:,:])
            with h5py.File(os.path.join(root, 'kinova_cartesian_states.h5'), 'r') as f:
                # state = np.concatenate((f['positions'][()], f['orientations'][()]), axis=1) # TODO: Will need to probs change this
                # self.kinova_states.append(state)
                self.kinova_states.append(f['positions'][()])

    def _get_tactile_image(self, tactile_value):
        tactile_image = get_tactile_image(tactile_value)
        return self.resize_transform(tactile_image)

    # tactile_values: (N,16,3) - N: number of sensors
    # robot_states: { allegro: allegro_tip_positions: (3*M,) - 3 values for each finger M: number of fingers included,
    #                 kinova: kinova_states : (3,) - cartesian position of the arm}
    def _get_one_representation(self, tactile_values, robot_states):
        states = np.concatenate([robot_states[robot_type] for robot_type in self.robots], axis=0)
        assert len(states) == len(self.allegro_finger_indices) + 3, "len(states) in get_one_representation: {}".format(len(states))
        
        # For each tactile value get the tactile image
        if self.only_states:
            return states

        if self.use_encoder:
            tactile_image = self._get_tactile_image(tactile_values).unsqueeze(dim=0)
            tactile_repr = self.encoder(tactile_image)
            tactile_repr = tactile_repr.detach().cpu().numpy().squeeze() # Remove the axes with dimension 1 - shape: (64,)
        else:
            tactile_repr = tactile_values.flatten() # This will have shape (96,)
        return np.concatenate((tactile_repr, states), axis=0)

    def _get_all_representations(
        self
    ):  
        print('Getting all representations')
        # For each tactile value and allegro tip position 
        # get one representation and add it to all representations
        if self.use_encoder:
            repr_dim = self.cfg.encoder.out_dim
        else:
            repr_dim = len(self.sensor_indices) * 16 * 3
        if 'allegro' in self.robots:  repr_dim += len(self.allegro_finger_indices)
        if 'kinova' in self.robots: repr_dim += 3

        self.all_representations = np.zeros((
            len(self.tactile_indices), repr_dim
        ))

        print('all_representations.shape: {}'.format(self.all_representations.shape))
        pbar = tqdm(total=len(self.tactile_indices))

        for index in range(len(self.tactile_indices)):
            demo_id, tactile_id = self.tactile_indices[index]
            _, allegro_tip_id = self.allegro_indices[index]
            _, kinova_id = self.kinova_indices[index]

            tactile_value = self.tactile_values[demo_id][tactile_id] # This should be (N,16,3)
            allegro_tip_position = self.allegro_tip_positions[demo_id][allegro_tip_id] # This should be (M*3,)
            kinova_state = self.kinova_states[demo_id][kinova_id]
            
            robot_states = dict(
                allegro = allegro_tip_position,
                kinova = kinova_state
            )
            representation = self._get_one_representation(
                tactile_value, 
                robot_states
            )
            # Add only tip positions as the representation
            self.all_representations[index, :] = representation[:]
            pbar.update(1)

        pbar.close()

    # tactile_values.shape: (16,15,3)
    # robot_state: {allegro: allegro_joint_state (16,), kinova: kinova_cart_state (7,)}
    def get_action(self, tactile_values, recv_robot_state, visualize=False):
        # Get the allegro tip positions with kdl solver 
        allegro_joint_state = recv_robot_state['allegro']
        fingertip_positions = self.kdl_solver.get_fingertip_coords(allegro_joint_state) # - fingertip position.shape: (12)
        
        if 'kinova' in self.robots:
            kinova_cart_state = recv_robot_state['kinova']

        # Get the tactile image from the tactile values
        curr_tactile_values = tactile_values[self.sensor_indices,:,:]
        curr_fingertip_position = fingertip_positions[self.allegro_finger_indices]

        print('curr_tactile_values.shape: {}, curr_fingertip_position.shape: {}, curr_kinova_state.shape: {}'.format(
            curr_tactile_values.shape, curr_fingertip_position.shape, kinova_cart_state.shape
        ))

        assert curr_tactile_values.shape == (len(self.sensor_indices),16,3) and curr_fingertip_position.shape == (len(self.allegro_finger_indices),)

        curr_robot_state = dict(
            allegro = curr_fingertip_position,
            kinova = kinova_cart_state
        )
        # Get the representation with the given tactile value
        curr_representation = self._get_one_representation(
            curr_tactile_values, 
            curr_robot_state
        )

        # k = 20
        # if self.only_states:
        #     only_tip_representation = np.zeros_like(curr_representation)
        #     only_tip_representation[-6:] = curr_representation[-6:]
        #     nn_idxs = self._get_knn_idxs(only_tip_representation, k=k) # TODO: Fix this
        # else:
        #     nn_idxs = self._get_knn_idxs(curr_representation, k=k)

        _, nn_idxs = self.knn.get_k_nearest_neighbors(curr_representation, k=self.nn_k)
        print('nn_idxs: {}'.format(nn_idxs))

        # Choose the action with the buffer 
        id_of_nn = self.buffer.choose(nn_idxs)
        nn_id = nn_idxs[id_of_nn]
        print('chosen nn_id: {}'.format(nn_id))
        demo_id, action_id = self.allegro_action_indices[nn_id] 
        nn_allegro_action = self.allegro_actions[demo_id][action_id+1] # Get the next commanded action (commanded actions are saved in that timestamp)
        nn_action = dict(
            allegro = nn_allegro_action
        )
        
        if 'kinova' in self.robots:
            _, kinova_id = self.kinova_indices[nn_id] 
            nn_kinova_action = self.kinova_states[demo_id][kinova_id+1] # Get the next saved kinova_state
            print('kinova action shape: {}'.format(nn_kinova_action.shape))
            assert nn_kinova_action.shape == (3,), 'kinova_action shape : {} incorrect - it shouldve been (3,)'.format(nn_kinova_action.shape)
            kinova_action = np.zeros(6)
            kinova_action[:3] = nn_kinova_action - kinova_cart_state # Since we're only doing velocity control
            nn_action['kinova'] = kinova_action

        # Visualize if given 
        if visualize: # TODO: This should be fixed for the whole robot
            self._visualize_state(
                curr_tactile_values, 
                curr_fingertip_position,
                nn_id
            )

        print('nn_action: {}'.format(nn_action))
        self.state_id += 1

        return nn_action

In [13]:
out_dir = '/home/irmak/Workspace/tactile-learning/tactile_learning/out/2023.01.02/19-29_byol_bs_1028_box_handle_lifting'
vinn = DeployVINN(out_dir)

self.allegro_finger_indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
self.only_states: False, self.use_encoder: True
Getting all representations
all_representations.shape: (72266, 79)


100%|██████████| 72266/72266 [00:43<00:00, 1660.80it/s]
/home/irmak/miniconda3/envs/tactile_learning/lib/python3.9/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
/home/irmak/miniconda3/envs/tactile_learning/lib/python3.9/site-packages/ikpy/chain.py:60: UserWarning: Link joint_3.0_tip (index: 5) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any tra

In [14]:
x = np.random.rand(15,16,3)
allegro_joint_states = np.random.rand(16)
kinova_cart_states = np.random.rand(3)
robot_state = dict(
    allegro = allegro_joint_states,
    kinova = kinova_cart_states
)

In [15]:
action = vinn.get_action(x, robot_state)

curr_tactile_values.shape: (15, 16, 3), curr_fingertip_position.shape: (12,), curr_kinova_state.shape: (3,)
nn_idxs: [37896 37897 37895 37894 37901 14827 14829 14828 15566 15567 35652 14832
 14831 14830 35659 35657 35658 35661 35660 35654]
chosen nn_id: 37896
kinova action shape: (3,)
nn_action: {'allegro': array([ 0.        , -0.17453294,  0.7853982 ,  0.7853982 ,  0.        ,
       -0.17453294,  0.7853982 ,  0.7853982 ,  0.08726646, -0.08726646,
        0.87266463,  0.7853982 ,  1.0471976 ,  0.43633232,  0.2617994 ,
        0.7853982 ], dtype=float32), 'kinova': array([-1.19047608,  0.01418956, -0.3219771 ,  0.        ,  0.        ,
        0.        ])}
